# gdptools CONUS404 Spatial Aggregation over DRB-extent HUC12s

This tutorial demonstrates the use of gdptools, a python package for area-weighted interpolation of *source* gridded datasets, such as conus404, to *target* polygonal geospatial fabrics.  Source datasets can be any gridded dataset that can be opened in XArray.  However it's important to note that gdptools, operations on XArray Datasets or DataArrays with dimensions of (Y,X,Time) generally.  As such climate datasets that have ensemble dimensions will require subsetting by ensemble to obtain the a dataset with the proper dimensions.  The target dataset can be any polygonal dataset that can be read by GeoPandas.  GDPtools also has capabilities of interpolating gridded data to lines as well, but our focus here is interpolating to polygons. 

In this workflow, CONUS404 is aggregated to Deleware River Basin (DRB) HUC12s. The spatial polygons used in this notebook come from the [**NHDPlusV2 snapshot of the Watershed Boundary Dataset HUC12 boundaries**](https://www.sciencebase.gov/catalog/item/60cb5edfd34e86b938a373f4) provided through the [PyGeoHydro](https://docs.hyriver.io/readme/pygeohydro.html) python package.

We use the HyTest intake catalog to access CONUS404 from the OSN pod. This notebook provides a relatively simple and efficient workflow that can be easily run on a local computer.

In [ ]:
# Common python packages
import xarray as xr
import hvplot.xarray
import hvplot.pandas
import hvplot.dask
import intake
import warnings
import intake_xarray
import datetime
import holoviews as hv
import geoviews as gv
from holoviews import opts
import cartopy.crs as ccrs
import panel as pn
import numpy as np
import pandas as pd
import geopandas as gpd

# HyRiver packages
from pynhd import NLDI, WaterData
import pygeohydro as gh
# GDPTools packages
from gdptools import AggGen, UserCatData, WeightGen
import os
# Until gdptools updates it's numpy dependency to v2, the environment statement below is required
os.environ["HYRIVER_CACHE_DISABLE"] = "true"

hv.extension("bokeh")
pn.extension()

warnings.filterwarnings('ignore')

Here we setup a variable the sets our local context, working on the HPC or working locally on your Desktop.  This just modifies the access point of the conus404 data, using the Hovenweep access for HPC and the OSN pod access for the Desktop.

In [ ]:
t_sys = "Desktop"  # "HPC"  # or "Desktop"


We can use a subset of the [**HyRiver**](https://docs.hyriver.io/index.html) Python packages to access HUC12 geometries representing the Delaware River Basin. The process involves several steps:

1. **Select the HUC4 Mid-Atlantic Region**:
    - This region encompasses the Delaware River Basin (HUC4 code: 0204).

2. **Retrieve HUC12 Basins within the Selected HUC4**:
    - Obtain all HUC12 basins that fall within the HUC4 Mid-Atlantic region.

3. **Filter HUC12 Basins**:
    - Focus on the HUC12 basins within the two HUC6 regions whose drainages terminate in the Delaware River Basin (DRB).
    - Exclude basins with drainages that terminate at the coast.

We used [**Science in Your Watershed**](https://water.usgs.gov/wsc/map_index.html) to help identify the HUC6 regions that drain directly into the DRB.

In [ ]:

wbd = gh.WBD("huc4")
del_huc4 = wbd.byids(field="huc4", fids="0204")
huc12_basins = WaterData('wbd12').bygeom(del_huc4.geometry[0])
filtered_gdf = huc12_basins[huc12_basins['huc12'].str.startswith(('020401', '020402'))]
filtered_gdf

In [ ]:
from holoviews.element.tiles import OSM
drb = filtered_gdf.hvplot(
    geo=True, coastline='50m', alpha=0.2,  frame_width=300,
    xlabel="longitude", ylabel="latitude",
    title="Delaware River HUC12 basins", aspect='equal'
)
OSM() * drb

Access conus404 via the HyTest intake catalog.

In [ ]:
# open the hytest data intake catalog
hytest_cat = intake.open_catalog("https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml")
list(hytest_cat)

In [ ]:
# open the conus404 sub-catalog
cat = hytest_cat['conus404-catalog']
list(cat)

There are a couple of options for accessing **conus404**:

1. **HPC Setting (`t_sys = HPC`)**:
    - **Assumption**: The notebook is run on the USGS HPC Hovenweep.
    - **Access Method**: Utilizes the on-premises version of the data.
    - **Benefits**:
        - **Workflow Association**: The workflow is directly linked to the data.
        - **Speed**: Eliminates the need to download data, significantly reducing access and processing time.

2. **Desktop Setting (`t_sys = Desktop`)**:
    - **Use Case**: Suitable for workflows that do not require HPC resources or for developing workflows locally before deploying them to the HPC.
    - **Access Method**: Connects to the **conus404** data via the OSN pod.
    - **Benefits**:
        - **Flexibility**: Allows for local development and testing.
        - **Performance**: Provides a fast connection to the data.


In [ ]:
## Select the dataset you want to read into your notebook and preview its metadata
if t_sys == "HPC":
    dataset = 'conus404-daily-diagnostic-onprem-hw'
elif t_sys == "Desktop":
    dataset = 'conus404-daily-diagnostic-osn' 
else:
    print("Please set the variable t_sys above to one of 'HPC' or 'Desktop'")        
cat[dataset]

In [ ]:
# read in the dataset and use metpy to parse the crs information on the dataset
print(f"Reading {dataset} metadata...", end='')
ds = cat[dataset].to_dask().metpy.parse_cf()
ds

### GDPTools Background

In this section, we utilize three data classes from the `gdptools` package: `UserCatData`, `WeightGen`, and `AggGen`.

* [**UserCatData**](https://gdptools.readthedocs.io/en/develop/user_input_data_classes.html):  
  Serves as a data container for both the source and target datasets, along with their associated metadata. The instantiated object `user_data` is employed by both the `WeightGen` and `AggGen` classes.

* [**WeightGen**](https://gdptools.readthedocs.io/en/develop/weight_gen_classes.html):  
  Responsible for calculating the intersected areas between the source and target datasets. It generates normalized area-weights, which are subsequently used by the `AggGen` class to compute interpolated values between the datasets.

* [**AggGen**](https://gdptools.readthedocs.io/en/develop/agg_gen_classes.html):  
  Facilitates the interpolation of target data to match the source data using the areal weights calculated by `WeightGen`. This process is conducted over the time period specified in the `UserCatData` object.

### Instantiation of the `UserCatData` class.

In [ ]:
# Coordinate Reference System (CRS) of the conus404 dataset
source_crs = ds.crs.crs_wkt

# Coordinate names of the conus404 dataset
x_coord = "x"
y_coord = "y"
t_coord = "time"

# Time period of interest for areal interpolation of conus404 to DRB HUC12s
# using the AggGen class below. Note: The dates follow the same format as the
# time values in the conus404 dataset.
sdate = "1979-10-01T00:00:00.000000000"
edate = "2022-10-01T00:00:00.000000000"

# Variables from the conus404 dataset used for areal interpolation
variables = ["T2MIN", "T2MAX", "RAINNCVMEAN"]

# CRS of the DRB HUC12 polygons
target_crs = 5070

# Column name for the unique identifier associated with target polygons.
# This ID is used in both the generated weights file and the areal interpolated output.
target_poly_idx = "huc12"

# Common equal-area CRS for reprojecting both source and target data.
# This CRS is used for calculating areal weights in the WeightGen class.
weight_gen_crs = 5070

# Instantiate the UserCatData class, which serves as a container for both
# source and target datasets, along with associated metadata. The UserCatData
# object provides methods used by the WeightGen and AggGen classes to subset
# and reproject the data.
user_data = UserCatData(
    ds=ds,
    proj_ds=source_crs,
    x_coord=x_coord,
    y_coord=y_coord,
    t_coord=t_coord,
    var=variables,
    f_feature=filtered_gdf,
    proj_feature=target_crs,
    id_feature=target_poly_idx,
    period=[sdate, edate],
)


### Weight Generation with `WeightGen`

In this section, we utilize the `WeightGen` class from the `gdptools` package to calculate the normalized areal weights necessary for interpolating the source gridded data (`conus404`) to the target polygonal boundaries (`DRB HUC12s`). The areal weights represent the proportion of each grid cell that overlaps with each polygon, facilitating accurate **areal interpolation** of the data. These weights are calculated using the `calculate_weights()` method.

**Weight Calculation Process:**

1. **Subset Source Data**: The source data is subset based on the bounds of the target data, with an additional small buffer to ensure coverage. The buffer size is determined based on [specific criteria or methodology].

2. **Create cell boundary GeoDataFrame**: A GeoDataFrame of the cell boundaries is created for each node in the subsetted source data, enabling spatial operations.

3. **Validate Geometries**: The target file is checked for invalid geometries, which can occur due to various reasons such as topology errors. Invalid geometries are fixed using Shapely's `make_valid()` method to prevent failures during intersection calculations.

4. **Calculate and Normalize Areas**: For each polygon, `gdptools` calculates the area of each intersecting grid cell and normalizes it by the total area of the target polygon. This ensures that the weights for each polygon sum to 1, provided the polygon is entirely covered by the source data.
   
   - **Validation**: A quick check on the weights can be performed by grouping the resulting weights by the `target_poly_idx` and calculating the sum. For all polygons completely covered by the source data, the weights will sum to 1.

**Note:** The `method` parameter can be set to one of `"serial"`, `"parallel"`, or `"dask"`. Given the scale of the gridded `conus404` data (4 km × 4 km) and the number and spatial footprint of the `DRB HUC12s`, using `"serial"` in this case is the most efficient method. In subsequent sections, we will explore how the `"parallel"` and `"dask"` methods can provide speed-ups in the areal interpolation process, as well as in the computation of weights for broader CONUS-wide targets.


In [ ]:
%%time
wght_gen = WeightGen(
    user_data=user_data,
    method="serial",
    output_file="wghts_drb_ser_c404daily.csv",
    weight_gen_crs=6931
)

wdf = wght_gen.calculate_weights()

### Areal Interpolation with the `AggGen` Class

In this section, we demonstrate the use of the `AggGen` class and its `calculate_agg()` method from the `gdptools` package to perform areal interpolation. We will explore all three `agg_engine` options: `"serial"`, `"parallel"`, and `"dask"`. The following links provide detailed documentation on the available parameter options:

* [**agg_engines**](https://gdptools.readthedocs.io/en/develop/agg_gen_classes.html#gdptools.agg_gen.AGGENGINES)
* [**agg_writers**](https://gdptools.readthedocs.io/en/develop/agg_gen_classes.html#gdptools.agg_gen.AGGWRITERS)
* [**stat_methods**](https://gdptools.readthedocs.io/en/develop/agg_gen_classes.html#gdptools.agg_gen.STATSMETHODS)

When using `AggGen` and the `calculate_agg()` method, it is important to consider the overlap between the source and target data when selecting the `stat_method` parameter value. All statistical methods have a masked variant in addition to the standard method; for example, `"mean"` and `"masked_mean"`. In cases where the source data has partial overlap with a target polygon, the `"mean"` method will return a missing value for the polygon, whereas the `"masked_mean"` method will calculate the statistic based on the available overlapping source cells. These considerations help users determine whether using a masked statistic is desirable or if a missing value would be preferred, allowing for post-processing of missing values (e.g., using nearest-neighbor or other approaches to handle the lack of overlap). In the case here conus404 completely covers the footprint of the DRB HUC12s, as such the `"mean"` method would be sufficient.  
 

In [ ]:
%%time
agg_gen = AggGen(
    user_data=user_data,
    stat_method="mean",
    agg_engine="parallel",
    jobs=4,
    agg_writer="netcdf",
    weights='wghts_drb_ser_c404daily.csv',
    out_path='.',
    file_prefix="serial_weights",
    precision=8
)
ngdf, ds_out = agg_gen.calculate_agg()

### Output

The [**`calculate_agg()`**](https://gdptools.readthedocs.io/en/develop/agg_gen_classes.html#gdptools.agg_gen.AggGen.calculate_agg) method returns two objects: `ngdf` and `ds_out`.

- **`ngdf`**: A `GeoDataFrame` derived from the target GeoDataFrame (`filtered_gdf`) specified in the `UserCatData` container. This GeoDataFrame has been both sorted and dissolved based on the identifiers in the `"huc12"` column, as defined by the `target_poly_idx` parameter.
  
- **`ds_out`**: The areally weighted interpolation output as an XArray Dataset. The Dataset consists of dimensions `time` and `huc12`, and the data variables are `T2MIN`, `T2MAX`, and `RAINNCVMEAN`.  

A preview of the `ngdf` GeoDataFrame below shows that it is sorted by `"huc12"`. In this case, there are no duplicate `"huc12"` values, resulting in the original and output GeoDataFrames having the same number of rows.  Some target datasets such as the GFv1.1, will result in many dissolved geometries.  


In [ ]:
ngdf.head()

In [ ]:
ds_out

### Plot the results as a quick sanity check

Here we plot the results along with the corresponding conus404 values.  To make the plot a little more interesting we choose the time step with the most precipitation.  This provides a quick qualitative sanity check.  If one is intersted in looking in more detail in a graphic presentation of a target polygon, overlayed on the intersecting grid cells, with the grid-cell values and weights shown for each intersection, please look at the tail end of notebook [**ClimateR-Catalog - Terraclime data**](https://gdptools.readthedocs.io/en/develop/Examples/ClimateR-Catalog/terraclime_et.html) for some example code to generate a plot which give a more quantitative expression of the result. 

**Format the interpolated results for plotting**

In [ ]:
# Convert processed xarray Dataset to DataFrame
df = ds_out.to_dataframe().reset_index()
# Pivot the DataFrame to have variables as separate columns
df_pivot = df.pivot_table(index=['time', 'huc12'], values=['T2MAX', 'T2MIN', 'RAINNCVMEAN']).reset_index()

# Merge GeoDataFrame with DataFrame
merged_gdf = ngdf.to_crs(5070).merge(df_pivot, on='huc12')

# Convert RAINNCVMEAN from kg/m²/s to mm/day
merged_gdf['RAINNCVMEAN_mm_day'] = merged_gdf['RAINNCVMEAN'] * 86400
# Convert T2MAX and T2MIN from Kelvin to Celsius
merged_gdf['T2MAX_C'] = merged_gdf['T2MAX'] - 273.15
merged_gdf['T2MIN_C'] = merged_gdf['T2MIN'] - 273.15

# Calculate total precipitation for each time step
rain_sum = merged_gdf.groupby('time')['RAINNCVMEAN_mm_day'].sum()

# Identify the time step with the maximum total precipitation
max_rain_time = rain_sum.idxmax()
print(f"Time step with maximum total precipitation: {max_rain_time}")

# Subset the GeoDataFrame for the selected time step
subset = merged_gdf[merged_gdf['time'] == max_rain_time]


**Process the sub-setted conus404 data for plotting**

In [ ]:

# We can use our agg_gen object to retrieve the subsetted conus404 data
da_t2max = agg_gen.agg_data.get("T2MAX").da
da_t2min = agg_gen.agg_data.get("T2MIN").da
da_rain = agg_gen.agg_data.get("RAINNCVMEAN").da

# Get the subsetted raw conus404 cubes used in the areal interpolation
da_t2max = agg_gen.agg_data.get("T2MAX").da.sel(time=max_rain_time) - 273.15
da_t2min = agg_gen.agg_data.get("T2MIN").da.sel(time=max_rain_time) - 273.15
da_rain = agg_gen.agg_data.get("RAINNCVMEAN").da.sel(time=max_rain_time) * 86400

**Generate the plot**
* Here we use GeoViews and HoloViews

In [ ]:
# Define Cartopy CRS using EPSG code 5070 (NAD83 / Conus Albers)
crs_cartopy = ccrs.epsg(5070)

# Define color maps for each variable
color_maps = {
    'T2MAX_C': 'Reds',
    'T2MIN_C': 'Blues',
    'RAINNCVMEAN_mm_day': 'Greens'
}
# Define color maps for raw data
color_maps_raw = {
    'T2MAX': 'Reds',
    'T2MIN': 'Blues',
    'RAINNCVMEAN': 'Greens'
}
# Create Polygons for T2MAX_C
map_T2MAX = gv.Polygons(
    subset.to_crs(5070), 
    vdims=['T2MAX_C'],
    crs=crs_cartopy  # Use Cartopy CRS here
).opts(
    cmap=color_maps['T2MAX_C'],
    colorbar=True,
    tools=['hover'],
    title='T2MAX (°C)',  # Included units
    alpha=0.7,
    frame_width=200,
    aspect='equal',
    padding=0,
)

# Create Polygons for T2MIN_C
map_T2MIN = gv.Polygons(
    subset.to_crs(5070), 
    vdims=['T2MIN_C'],
    crs=crs_cartopy
).opts(
    cmap=color_maps['T2MIN_C'],
    colorbar=True,
    tools=['hover'],
    title='T2MIN (°C)',  # Included units
    alpha=0.7,
    frame_width=200,
    aspect='equal',
    padding=0,
)

# Create Polygons for RAINNCVMEAN_mm_day
map_RAINNCVMEAN = gv.Polygons(
    subset.to_crs(5070), 
    vdims=['RAINNCVMEAN_mm_day'],
    crs=crs_cartopy
).opts(
    cmap=color_maps['RAINNCVMEAN_mm_day'],
    colorbar=True,
    tools=['hover'],
    title='RAINNCVMEAN (mm/day)',  # Included units
    alpha=0.7,
    frame_width=200,
    aspect='equal',
    padding=0,
)

# Create raw sub-setted data frames
# Assuming 'lon' and 'lat' are the coordinate names; adjust if necessary
coord_x = 'x'  # Replace with actual x-coordinate name
coord_y = 'y'  # Replace with actual y-coordinate name

# Create HoloViews Image for T2MAX (Raw)
image_T2MAX_raw = hv.Image(
    da_t2max, 
    kdims=[coord_x, coord_y],
    vdims=['T2MAX']
).opts(
    cmap=color_maps_raw['T2MAX'],
    colorbar=True,
    tools=['hover'],
    title='T2MAX Raw (°C)',
    alpha=0.7,
    frame_width=200,
    aspect='equal'
)

# Create HoloViews Image for T2MIN (Raw)
image_T2MIN_raw = hv.Image(
    da_t2min, 
    kdims=[coord_x, coord_y],
    vdims=['T2MIN']
).opts(
    cmap=color_maps_raw['T2MIN'],
    colorbar=True,
    tools=['hover'],
    title='T2MIN Raw (°C)',
    alpha=0.7,
    frame_width=200,
    aspect='equal'
)

# Create HoloViews Image for RAINNCVMEAN (Raw)
image_RAIN_raw = hv.Image(
    da_rain, 
    kdims=[coord_x, coord_y],
    vdims=['RAINNCVMEAN']
).opts(
    cmap=color_maps_raw['RAINNCVMEAN'],
    colorbar=True,
    tools=['hover'],
    title='RAINNCVMEAN Raw (mm/day)',
    alpha=0.7,
    frame_width=200,
    aspect='equal'
)

# Create a GridSpec with 3 rows and 3 columns
grid = pn.GridSpec(ncols=3, nrows=2, sizing_mode='fixed', width=900, height=1200)

# Add title spanning all columns in the first row
title = pn.pane.Markdown(
    f"<h2 style='text-align: center;'>Areal Interpolation for {max_rain_time.strftime('%Y-%m-%d')}</h2>",
    width=900,
    height=20
)

# Add raw data plots in the first row
grid[0, 0] = image_T2MAX_raw
grid[0, 1] = image_T2MIN_raw
grid[0, 2] = image_RAIN_raw

# Add processed data plots in the second row
grid[1, 0] = map_T2MAX
grid[1, 1] = map_T2MIN
grid[1, 2] = map_RAINNCVMEAN


# Display the grid
# final_layout
grid


# Combine the three maps into a single layout arranged horizontally
maped_layout = pn.Row(
    map_T2MAX,
    pn.Spacer(sizing_mode="stretch_width"),
    map_T2MIN,
    pn.Spacer(sizing_mode="stretch_width"),
    map_RAINNCVMEAN,
    width=800
)

# Combine raw data plots horizontally
raw_layout = pn.Row(
    image_T2MAX_raw,
    pn.Spacer(sizing_mode="stretch_width"),
    image_T2MIN_raw,
    pn.Spacer(sizing_mode="stretch_width"),
    image_RAIN_raw,
    width=800
)

# Create a main title using HTML for center alignment within Markdown
title = pn.pane.Markdown(
    f"<h3 style='text-align: center;'>Areal Interpolation for {max_rain_time.strftime('%Y-%m-%d')}</h3>",
    # width=1800  # Adjust width as needed
)

# Combine both rows vertically
combined_layout = pn.Column(
    raw_layout,
    maped_layout
)
# Combine the title and the layout vertically
final_layout = pn.Column(
    title,
    combined_layout,
    width=850
)

final_layout

There is a clear increasing gradient in temperature from north to south that is visible in the interpolated results.  

### Parallel and Dask Methods

The domain of this workflow is small enough that using either the parallel or dask methods are not necessary.  However there is a speedup that we illustrate.  The parallel and dask engines used in the AggGen object operate in a similar manner using `multiprocessing` and `dask bag` respectivly. Using the jobs parameter the user can specify the number of processes to run.  The target data is chunked by the number of processes and each processor recieves a chunked GeoDataFrame along with a copy of the sub-setted source data.  This creates an overhead that can determine how effiently the parallel processing runs.

The tradeoff in using parallel processing lies in the balance between the number of processors and the overhead of copying data. While increasing the number of processors can significantly reduce computation time by dividing the workload, it also increases the amount of memory used for duplicate datasets and the coordination time between processes. There is a 'sweet spot' where the number of processors maximizes performance but beyond this point, additional processors may slow down the operation due to the overhead of managing more processes. The optimal number of processors depends on the size of the data, available memory, and system architecture, and can typically be found through experimentation.

Importantly, most of the time in processing here is dominated by downloading the data, so the speedup is relatively small.  For larger domains the processing will be a larger percentage of the total time and the speedup should be more pronounced.  Well explore that in the CONUS scale processing of conus404 on Hovenweep.

In [ ]:
%%time
gg_gen = AggGen(
    user_data=user_data,
    stat_method="masked_mean",
    agg_engine="parallel",
    agg_writer="netcdf",
    weights='wghts_drb_ser_c404daily.csv',
    out_path='.',
    file_prefix="testing_p2",
    precision=8,
    jobs=4
)
ngdf, ds_out = agg_gen.calculate_agg()

In [ ]:
%%time
agg_gen = AggGen(
    user_data=user_data,
    stat_method="masked_mean",
    agg_engine="dask",
    agg_writer="netcdf",
    weights='wghts_drb_ser_c404daily.csv',
    out_path='.',
    file_prefix="testing_p3",
    precision=8,
    jobs=4
)
ngdf, ds_out = agg_gen.calculate_agg()

In [ ]:
ds_agg = xr.open_dataset("testing_p3.nc")
ds_agg

In [ ]:
ds_agg.T2MAX.values